In [3]:
#pip install pyspark

    100% |████████████████████████████████| 215.7MB 157kB/s eta 0:00:01    57% |██████████████████▍             | 123.9MB 15.8MB/s eta 0:00:06    67% |█████████████████████▌          | 144.6MB 32.5MB/s eta 0:00:03
    100% |████████████████████████████████| 204kB 17.3MB/s ta 0:00:01
  Stored in directory: /Users/soumya_acharya/Library/Caches/pip/wheels/47/9b/57/7984bf19763749a13eece44c3174adb6ae4bc95b920375ff50
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [7]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
#sc = SparkContext('local')
spark = SparkSession(sc)

In [8]:
spark

In [9]:
sqlContext

In [10]:
if 'sc' not in locals():
    from pyspark.context import SparkContext
    from pyspark.sql.context import SQLContext
    from pyspark.sql.session import SparkSession
    
    sc = SparkContext()
    sqlContext = SQLContext(sc)
    spark = SparkSession(sc)

In [12]:
if 'sc' not in locals():
    from pyspark.context import SparkContext
    from pyspark.sql.context import SQLContext
    from pyspark.sql.session import SparkSession
    
    sc = SparkContext()
    sqlContext = SQLContext(sc)
    spark = SparkSession(sc)

In [14]:
def run_sql(statement):
    try:
        result = sqlContext.sql(statement)
    except Exception as e:
        print(e.desc, '\n', e.stackTrace)
        return
    return result

In [16]:
run_sql('drop database if exists country_club cascade')
run_sql('create database country_club')
#run_sql("CREATE DATABASE IF NOT EXISTS country_club LOCATION 'country_club.db'")
dbs = run_sql('show databases')
dbs.toPandas()
#spark.sql("show databases").show) 

,databaseName
0,country_club
1,default


In [17]:
# File location and type
file_location_bookings = "./Bookings.csv"
file_location_facilities = "./Facilities.csv"
file_location_members = "./Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

In [18]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()


Bookings Schema
root
 |-- bookid: integer (nullable = true)
 |-- facid: integer (nullable = true)
 |-- memid: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- slots: integer (nullable = true)

Facilities Schema
root
 |-- facid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- membercost: double (nullable = true)
 |-- guestcost: double (nullable = true)
 |-- initialoutlay: integer (nullable = true)
 |-- monthlymaintenance: integer (nullable = true)

Members Schema
root
 |-- memid: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- address: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- telephone: string (nullable = true)
 |-- recommendedby: integer (nullable = true)
 |-- joindate: timestamp (nullable = true)



In [24]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet")
#.write.mode(SaveMode.Overwrite)
.saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet")
#.write.mode(SaveMode.Overwrite)
.saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet")
#.write.mode(SaveMode.Overwrite)
.saveAsTable(permanent_table_name_members)


SyntaxError: invalid syntax (<ipython-input-24-366f47bbdc76>, line 3)

In [21]:
run_sql('use country_club')
run_sql('REFRESH table bookings')
run_sql('REFRESH table facilities')
run_sql('REFRESH table members')
tbls = run_sql('show tables')
tbls.toPandas()

,database,tableName,isTemporary
0,country_club,bookings,False
1,country_club,facilities,False
2,country_club,members,False


In [22]:
result = run_sql('''
                    SELECT * 
                    FROM bookings 
                    LIMIT 3
                 ''')
result.toPandas()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2


In [25]:
result = run_sql('''
                   SELECT * 
                    FROM facilities 
                    where membercost = 0.0
                 ''')
result.toPandas()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,2,Badminton Court,0.0,15.5,4000,50
1,3,Table Tennis,0.0,5.0,320,10
2,7,Snooker Table,0.0,5.0,450,15
3,8,Pool Table,0.0,5.0,400,15


In [29]:
#some text
#Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost?
#Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.


In [31]:
result = run_sql('''
                   SELECT * 
                    FROM facilities 
                    where membercost != 0.0 and membercost < (monthlymaintenance *20/100)
                 ''')
result.toPandas()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,4,Massage Room 1,9.9,80.0,4000,3000
3,5,Massage Room 2,9.9,80.0,4000,3000
4,6,Squash Court,3.5,17.5,5000,80


In [33]:
#Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.¶


In [34]:
result = run_sql('''
                   SELECT * 
                    FROM facilities 
                    where facid in (1,5)
                 ''')
result.toPandas()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25.0,8000,200
1,5,Massage Room 2,9.9,80.0,4000,3000


In [35]:
#Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100?
#Return the name and monthly maintenance of the facilities in question.

In [55]:
result = run_sql('''
                    SELECT 
                        CASE 
                          WHEN (monthlymaintenance > 100) THEN "expensive"
                          WHEN (monthlymaintenance < 100) THEN "cheap"
                          ELSE ""
                         END AS iscostly 
                    FROM facilities 
                 ''')
result.toPandas()

,iscostly
0,expensive
1,expensive
2,cheap
3,cheap
4,expensive
5,expensive
6,cheap
7,cheap
8,cheap


In [64]:
#You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solutio

result = run_sql('''
                    SELECT firstname, surname
                    FROM members 
                     where joindate in (
                     SELECT max(joindate)
                     FROM  from members 
                     )
                    
                 ''')
result.toPandas()

,firstname,surname
0,Darren,Smith


In [87]:
#Q7: How can you produce a list of all members who have used a tennis court?
#Include in your output the name of the court, and the name of the member formatted as a single column.
#Ensure no duplicate data
#Also order by the member name.

result = run_sql('''
                    SELECT  distinct m.firstname, m.surname , m.memid as memberID,  b.facid as facilityId , f.name
                    from members as m
                    join bookings as b 
                    on m.memid = b.memid
                    join facilities as f 
                    on b.facid = f.facid
                     and b.facid = 1 order by m.firstname
                 ''')
result.toPandas()



,firstname,surname,memberID,facilityId,name
0,Anne,Baker,12,1,Tennis Court 2
1,Burton,Tracy,6,1,Tennis Court 2
2,Charles,Owen,10,1,Tennis Court 2
3,Darren,Smith,1,1,Tennis Court 2
4,David,Jones,11,1,Tennis Court 2
5,David,Farrell,28,1,Tennis Court 2
6,Florence,Bader,15,1,Tennis Court 2
7,GUEST,GUEST,0,1,Tennis Court 2
8,Gerald,Butters,5,1,Tennis Court 2
9,Henrietta,Rumney,27,1,Tennis Court 2


In [84]:
result = run_sql('''
                    SELECT * 
                    from facilities
                    
                 ''')
result.toPandas()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15
